In [7]:
import pandas as pd
import numpy as np

folder = '../trictrac_database'

In [8]:
# avis_clean = pd.read_csv(f'{folder}/avis_clean.csv', header=None, names=["Game id", "User id", "Game name UI", "Username", "Datetime", "Rating", "Comment title", "Comment body"]).drop_duplicates()
# users = pd.read_csv(f'{folder}/users.csv', header=None, names = ["Username", "User id"]).drop_duplicates()
# avis_clean["Datetime"] = pd.to_datetime(avis_clean["Datetime"]) # convert from object to datetime
jeux_clean = pd.read_csv(f'{folder}/jeux_clean.csv', header=None, names = ["Game id", "Game name website", "Game name UI", "Game name year", "Description", "Type", "Extra"])
jeux_clean[jeux_clean["Game id"] == 99]


,Game id,Game name website,Game name UI,Game name year,Description,Type,Extra
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,2 à 10|10 ans et +|30


In [9]:

# Separate extra info (number of player, age categories, game duration), replace ~ by NaN
jeux_clean[["Number of players", "Age", "Game duration"]] = jeux_clean.pop("Extra").str.split('|',n=3, expand=True, regex=False).replace('~', np.nan)
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Number of players,Age,Game duration
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,2 à 10,10 ans et +,30


In [10]:
# Number of players separation
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"jusqu\'à (\d+)", r"1 à \1", regex=True) # 'jusqu'à num' to '1 à num'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"à partir de (\d+)", r"\1 à 0", regex=True) # 'à partir de num' to 'num à 0'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"^\s*(\d+)\s*$", r"\1 à \1", regex=True) # 'num' to 'num à num'

jeux_clean[["Min number of players", "Max number of players"]] = jeux_clean.pop("Number of players").str.extract(r"(\d+)\s*à\s*(\d+)")
# Set types
jeux_clean["Min number of players"] = jeux_clean["Min number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].replace(0, np.nan,  regex=False)
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Age,Game duration,Min number of players,Max number of players
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,10 ans et +,30,2,10


In [11]:
jeux_clean

,Game id,Game name website,Game name UI,Game name year,Description,Type,Age,Game duration,Min number of players,Max number of players
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,8 ans et +,20,2,6
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,14 à 120,NaN,1,6
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,12 ans et +,240,2,4
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,10 ans et +,30,4,6
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,12 ans et +,150,3,6
...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,12 ans et +,20,2,4
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,8 ans et +,60,2,6
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",8 ans et +,25,2,6
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,NaN,15,1,1


In [12]:
# Age sepration
jeux_clean["Age"] = jeux_clean["Age"].replace(r"(\d+) ans et \+", r"\1 à 99", regex=True) # max age is fixed to 99
jeux_clean[["Age min", "Age max"]] = jeux_clean.pop("Age").str.extract(r"(\d+) à (\d+)")

In [13]:
# Type conversion
jeux_clean["Age max"] = jeux_clean["Age max"].astype('Int64')
jeux_clean["Age min"] = jeux_clean["Age min"].astype('Int64')
jeux_clean["Game duration"] = jeux_clean["Game duration"].astype('Int64')
jeux_clean.loc[jeux_clean["Age max"] > 99, "Age max"] = 99
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,30,2,10,10,99


In [14]:
print(jeux_clean.shape)
jeux_clean.drop_duplicates()


(10549, 11)


,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [15]:
jeux_clean.to_csv('../database_cleaned/jeux_clean.csv')

In [16]:
# separate number > 10 into first digit and the remainning ones
def digit_sep(n):
    nb_dig = int(np.log10(n))
    p = np.power(10, nb_dig)
    return n // p, n % p

condition = (jeux_clean["Min number of players"] >= 12) & (jeux_clean["Max number of players"].isna())

#for v in jeux_clean.loc[condition, "Min number of players"]:
    #print(v, digit_sep(v))
#jeux_clean_2 = jeux_clean.copy()
jeux_clean.loc[condition, ["Min number of players", "Max number of players"]] = jeux_clean.loc[condition, "Min number of players"].apply(lambda v : digit_sep(v)).to_list()
jeux_clean.to_csv("../database_cleaned/jeux_clean.csv")

In [17]:
jeux_clean

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [18]:
# jeux_clean_2.sort_values(by=["Min number of players"], ascending=False)
jeux_clean_2[jeux_clean["Min number of players"] > 12]
#print(jeux_clean.sort_values(by=["Min number of players"], ascending=False))
#jeux_clean[jeux_clean["Game id"] == 7595]

NameError: name 'jeux_clean_2' is not defined

In [93]:
jeux_clean.sort_values(by=["Min number of players"], ascending=False)[:30]
jeux_clean[(jeux_clean["Min number of players"] >= 10) & (~jeux_clean["Max number of players"].isna())]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
4027,4027,hollywood-lives,Hollywood Lives,Hollywood Lives (2005),Le jeu se déroule à l'âge d'or d'Hollywood (an...,Jeux de rôles,240,10,25,<NA>,<NA>


In [128]:
from collections import Counter
categories = [cat for cats in jeux_clean["Type"][~jeux_clean["Type"].isna()].str.split('|') for cat in cats]
count_cats = Counter(categories)
count_cats

Counter({'Jeux de plateau': 3163,
         'Jeux de cartes': 1742,
         'Jeux de guerre': 444,
         'Affrontement': 427,
         'Autres': 397,
         'Hasard (Dé, Cartes, ...)': 363,
         'Casse-tête': 297,
         'Placement': 285,
         'Combinaison': 263,
         'Déplacement': 257,
         'Ambiance': 250,
         'Jeux de pions': 244,
         'Médiéval-fantastique': 239,
         'Dés': 232,
         'Guerre': 226,
         'Gestion': 212,
         'Coopération': 210,
         'Gestion de main': 204,
         'Exploration': 201,
         'Animaux': 185,
         'Jeux de figurines': 185,
         'Jeu de carte à collectionner': 184,
         'Jeux de connaissances': 180,
         'Bluff': 179,
         'Fantastique': 172,
         'Majorité': 161,
         'Jeux de dés': 160,
         'Jeu de logique': 157,
         'Prise de risque': 153,
         'Science fiction': 153,
         'Commerce': 153,
         'Collection / famille': 150,
         'Médiéval': 1

In [124]:
id_games = jeux_clean[["Game id", "Game name UI"]].groupby(by="Game name UI").count()
id_games[id_games["Game id"] > 1]

jeux_clean[jeux_clean["Game name UI"] == "Waterloo"]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
10089,10089,waterloo-0,Waterloo,Waterloo (1981),Waterloo fait partie de la série International...,Jeux de guerre,240,2,3,12,99
10090,10090,waterloo-2,Waterloo,Waterloo (1996),Waterloo est la version française du jeu L'Arm...,Jeux de guerre,300,1,2,14,99
10091,10091,waterloo,Waterloo,Waterloo (2003),Idée du jeu : Waterloo : la dernière bataille ...,Jeux de guerre,60,2,3,12,99
10092,10092,waterloo-3,Waterloo,Waterloo (2009),Waterloo est le volume 4 de la Treefrog Line d...,Jeux de guerre,180,1,2,12,99


In [123]:
id_games[id_games["Game id"] > 1]


,Game id
Game name UI,
1830,2
1870,2
Age of Conan,2
Alcatraz,3
Alcazar,2
...,...
Warhammer : Rois des Tombes,2
Waterloo,4
Word-Whiz,2


In [19]:
jeux_clean["Age min"].describe()

count      9952.0
mean     9.532255
std      3.089553
min           2.0
25%           8.0
50%          10.0
75%          12.0
max         120.0
Name: Age min, dtype: Float64

In [ ]:
jeux_clean.loc[jeux_clean["Min number of players"].isna() & jeux_clean["Max number of players"].isna()][:20]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
20,20,1000e-le-point,1000e Le Point,1000e Le Point (1991),1000 questions sur la littérature française et...,Jeux de connaissances,60,<NA>,<NA>,12,99
26,26,12-jeux-et-leur-support,12 jeux et leur support,12 jeux et leur support (1987),Aucune description,Jeux de pions,<NA>,<NA>,<NA>,<NA>,<NA>
142,142,h-social-2-0,@h Social 2.0,@h Social 2.0 (2012),"Outil d'animation, il permet d'aborder de mani...",Jeux de cartes,<NA>,<NA>,<NA>,7,99
186,186,at-43-army-box-oni,AT 43 Army Box : ONI,AT 43 Army Box : ONI (2009),Composition de la compagnie ONI fournie dans l...,Jeux de figurines,60,<NA>,<NA>,14,99
312,312,agricola-the-legen-dary-forest-deck,Agricola : The legen*dary Forest-Deck,Agricola : The legen*dary Forest-Deck (2010),Une extension à Agricola composée de 24 cartes...,Jeux de cartes,<NA>,<NA>,<NA>,<NA>,<NA>
557,557,arkham-horror-investigator-pack-harvey-walters...,Arkham Horror : Investigator Pack Harvey Walte...,Arkham Horror : Investigator Pack Harvey Walte...,Des figurines pour Arkham Horror,Jeux de plateau,<NA>,<NA>,<NA>,<NA>,<NA>
686,686,auction,Auction,Auction,Aucune description,Jeux de cartes,10,<NA>,<NA>,12,99
711,711,autorace,Autorace,Autorace,Aucune description,Jeux de plateau,<NA>,<NA>,<NA>,<NA>,<NA>
728,728,aventure-et-fortune,Aventure et Fortune,Aventure et Fortune,Un jeu commandité par une banque? On trouve de...,Jeux de plateau,<NA>,<NA>,<NA>,<NA>,<NA>
902,902,battlelore-horde-horrifique,BattleLore : Horde Horrifique,BattleLore : Horde Horrifique (2010),Horde Horrifique est une extension pour Battle...,Jeux de plateau,<NA>,<NA>,<NA>,<NA>,<NA>
